In [5]:
file_path = "../.data/mix_model/data/all_raretrible.txt"
# file_path = "/home/azhao/projects/ai/train/liubin/mix_model/data/all_raretrible.txt"
data = open(file_path, "r").read().split("\n")
data = [x.strip().split(",") for x in data if len(x) > 0]
data = [[x[0], ",".join(x[1:]), "", "", ""] for x in data]
print(data[0:2])

[['感激', '好的,那我谢谢您了。', '', '', ''], ['中性', '没有词汇课的优惠券吗?', '', '', '']]


In [6]:
import pandas as pd

# 转换 data 到 pandas DataFrame
df = pd.DataFrame(data, columns=["label", "text", "pred_label", "score", "keywords"])

# 保存为 CSV 文件
cvs_file = "../.data/data.csv"
df.to_csv(cvs_file, index=False)

In [7]:
from dspy.datasets import DataLoader

# 创建 dspy.datasets 对象
dl = DataLoader()
# fields 选择指定列 input_keys
all_dataset = dl.from_csv(
    cvs_file,
    fields=("label", "text", "score", "pred_label", "score", "keywords"),
    input_keys=("text",),
)
print(all_dataset[0:2])

Generating train split: 0 examples [00:00, ? examples/s]

[Example({'label': '感激', 'text': '好的,那我谢谢您了。', 'score': None, 'pred_label': None, 'keywords': None}) (input_keys={'text'}), Example({'label': '中性', 'text': '没有词汇课的优惠券吗?', 'score': None, 'pred_label': None, 'keywords': None}) (input_keys={'text'})]


In [8]:
from collections import defaultdict
import random


def stratified_sample(from_dataset, label_attr, *dataset_sizes):
    label_groups = defaultdict(list)
    for item in from_dataset:
        label = getattr(item, label_attr)
        label_groups[label].append(item)

    result_sets = [[] for _ in dataset_sizes]

    for label, items in label_groups.items():
        random.shuffle(items)
        label_sizes = [size // len(label_groups) for size in dataset_sizes]
        start = 0
        for i, size in enumerate(label_sizes):
            end = start + size
            result_sets[i].extend(items[start:end])
            start = end

    all_items = [item for items in label_groups.values() for item in items]
    for i, (result_set, target_size) in enumerate(zip(result_sets, dataset_sizes)):
        shortage = target_size - len(result_set)
        if shortage > 0:
            result_sets[i].extend(random.sample(all_items, shortage))

    return result_sets


In [9]:
from collections import Counter

train_set, test_set, sample_set = stratified_sample(all_dataset, "label", 20, 50, 100)
for dataset in [train_set, test_set, sample_set]:
    label_counts = Counter(item.get("label") for item in dataset)
    print(label_counts)

Counter({'中性': 5, '高兴': 3, '抱怨': 3, '生气': 3, '感激': 2, '焦急': 2, '惊讶': 2})
Counter({'感激': 8, '中性': 7, '焦急': 7, '惊讶': 7, '高兴': 7, '抱怨': 7, '生气': 7})
Counter({'中性': 16, '感激': 14, '焦急': 14, '惊讶': 14, '高兴': 14, '抱怨': 14, '生气': 14})


In [ ]:
for ex in sample_set:
    print(ex.get("label"), ex.get("text"))

## base qa

In [13]:
import os

import dspy
from dotenv import find_dotenv, load_dotenv

load_dotenv(find_dotenv(), override=True)
api_key = os.getenv("ONEAPI_API_KEY")
base_url = os.getenv("ONEAPI_BASE_URL")

# 测试要加 model_type ，不然因 model 不存在，会默认 text
turbo = dspy.OpenAI(
    api_key=api_key,
    model="glm-4-flash",
    api_base=base_url,
    model_type="chat",
    max_tokens=8000,
)

dspy.settings.configure(
    lm=turbo,
)

In [14]:
import dspy
from dsp import passages2text


class BasicQA(dspy.Signature):
    """sentiment analysis from text"""

    question = dspy.InputField(desc="list of input text", format=passages2text)
    answer = dspy.OutputField(
        desc="question 中每一个输入`text`的情绪分类和评分(0-1.0),格式如 惊讶,0.7 抱怨,0.9 情绪分类包括 ['中性', '惊讶', '感激', '抱怨', '焦急', '生气', '高兴']",
        type=list,
    )

In [ ]:
# Define the predictor.
generate_answer = dspy.Predict(BasicQA)

samples = [x.text for x in sample_set[0:20]]
# Call the predictor on a particular input.
pred = generate_answer(question=samples)

# Print the input and the prediction.
print(f"Question: {samples}")
print(f"Predicted Answer:\n{pred.answer}")